# Model Evaluation & Interpetation
This notebook is used for:
1. [Model Evaluation](#1.-Model-Evaluation)
2. [Model Interpretation](#2.-Model-Interpretation)

### Declaring Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.colors import LogNorm, Normalize
import sklearn
import time
import datetime
import joblib
import warnings
from sklearn.model_selection import ParameterGrid
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

### Color Palette & Typeface Sizing

In [2]:
YELLOW = '#F2DC5D'
GREEN = '#9BC53D'
DARK_GREEN = '#597222'
RED = '#C3423F'
LIGHT_BLUE = '#2596BE'
GRAY = '#666666'
WHITE = '#FFFFFF'

AXIS_SIZE = 12
TITLE_SIZE = 16
DESCRIPTION_SIZE = 9
FIGURE_SIZE = (10*2/3,6*2/3)
EXPORT_DPI = 400

RANDOM_STATE = 14

### Import Dataset

In [3]:
df = pd.read_csv('../data/final.csv', dtype={'citizen': 'string', 'sex': 'string', 'age': 'string', 'decision': 'string', 'geo': 'string', 'TIME_PERIOD': 'string', 'GENCONV': "Int64", 'HUMSTAT': "Int64", 'SUB_PROT': "Int64", 'REJECTED': "Int64", 'TOTAL_POS': "Int64", 'TOTAL_APPS': "Int64", "POS_RATE": "Float64"}, keep_default_na=False, na_values=['nan'])

##remove partial 2023-Q3 Data
df = df[df["TIME_PERIOD"] != "2023-Q3"]

df

,citizen,sex,age,geo,TIME_PERIOD,GENCONV,HUMSTAT,SUB_PROT,REJECTED,TOTAL_POS,TOTAL_APPS
0,AD,F,UNK,AT,2008-Q1,0,0,0,0,0,0
1,AD,F,UNK,AT,2008-Q2,0,0,0,0,0,0
2,AD,F,UNK,AT,2008-Q3,0,0,0,0,0,0
3,AD,F,UNK,AT,2008-Q4,0,0,0,0,0,0
4,AD,F,UNK,AT,2009-Q1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7221109,ZW,UNK,Y_LT14,UK,2019-Q3,0,0,0,0,0,0
7221110,ZW,UNK,Y_LT14,UK,2019-Q4,0,0,0,0,0,0
7221111,ZW,UNK,Y_LT14,UK,2020-Q1,0,0,0,0,0,0
7221112,ZW,UNK,Y_LT14,UK,2020-Q2,0,0,0,0,0,0


## 1. Model Evaluation

### Replicate Splitting from `model-development.ipynb`

In [6]:
TARGET_VAR = "TOTAL_POS"

y = df_lagged[TARGET_VAR]
X = df_lagged.drop(['GENCONV', 'HUMSTAT', 'SUB_PROT', 'REJECTED', 'TOTAL_POS'], axis=1)

new_quarters = [q for q in quarters if q >= quarters[QUARTERS_OF_LAG]]
quarter_count = len(new_quarters) - 1

TRAIN_PORTION = 0.6
VAL_PORTION = 0.2
TEST_PORTION = 0.2

div_0 = new_quarters[0]
div_2 = new_quarters[int(quarter_count * (1 - TEST_PORTION))]
div_3 = new_quarters[quarter_count]

#seperate out test section
X_test = X[(div_2 <= X["TIME_PERIOD"]) & (X["TIME_PERIOD"] < div_3)]
y_test = y[(div_2 <= X["TIME_PERIOD"]) & (X["TIME_PERIOD"] < div_3)]

X_other = X[(div_0 <= X["TIME_PERIOD"]) & (X["TIME_PERIOD"] < div_2)]
y_other = y[(div_0 <= X["TIME_PERIOD"]) & (X["TIME_PERIOD"] < div_2)]

In [7]:
#****************************************************re-sort dataframe****************************************************

sort_order = ['citizen', 'sex', 'age', 'geo', 'TIME_PERIOD']
df = df.sort_values(by =sort_order) 

#*********************************************create sequential list of quarters****************************************************

quarters = []
for i in range(2008, 2024):
    quarters.append(str(i) + "-Q1")
    quarters.append(str(i) + "-Q2")
    quarters.append(str(i) + "-Q3")
    quarters.append(str(i) + "-Q4")

#****************************************************lagged features****************************************************

QUARTERS_OF_LAG = (4 * 1)

def add_lagged_features(df, features, maintained_columns, QUARTERS_OF_LAG):
    quarters = np.unique(df["TIME_PERIOD"])
    def lagged_features(target_var, lag_count, unit):
        lagged = pd.DataFrame()
        columns = []
        for i in range(1, lag_count + 1):
            lagged = pd.concat([lagged, target_var.shift(i)], axis=1)
            name = target_var.name
            if (i == 1):
                columns.append(name + " - lag " + str(i) + " " + str(unit))
            else:
                columns.append(name + " - lag " + str(i) + " " + str(unit) + "s")
        lagged.columns = columns
        return lagged.astype('Int64')

    #introduce lag for each feature
    df_lagged = df
    for f in features:
        df_lagged = pd.concat([df_lagged, lagged_features(df[f], QUARTERS_OF_LAG, "quarter")], axis=1)

    #remove all features with less than the lag amount of historical data
    #df_lagged = df_lagged[df_lagged.eq()]
    for i in range(0, QUARTERS_OF_LAG):
        shift_eq = df_lagged.eq(df_lagged.shift())
        keep = shift_eq[maintained_columns[0]]
        for j in range(1, len(maintained_columns)):
            keep = keep & shift_eq[maintained_columns[j]]
        df_lagged = df_lagged[keep]
        #print("lagged i" + str(i) + " of " + str(QUARTERS_OF_LAG))
    
    return df_lagged

df_lagged = add_lagged_features(df, ["TOTAL_POS", "TOTAL_APPS"], ['citizen', 'age', 'sex', 'geo'], QUARTERS_OF_LAG)

In [8]:
ordinal_ftrs = ['age', 'TIME_PERIOD']
ordinal_cats = [['UNK','Y_LT14','Y14-17','Y18-34','Y35-64','Y_GE65'], quarters]
                                                                     #^^i'm using quarters not new_quarters here so that
                                                                     #  the model can still tell where in history this q is
onehot_ftrs = ['citizen', 'geo', 'sex']
#onehot_ftrs = ['geo', 'sex']
minmax_ftrs = []
std_ftrs = [a for a in X.columns.to_list() if 'TOTAL' in a]
poly_ftrs = ['TOTAL_APPS', 'TOTAL_POS - lag 1 quarter', "TOTAL_APPS - lag 1 quarter"]

preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OrdinalEncoder(categories = ordinal_cats), ordinal_ftrs),
        ('onehot', OneHotEncoder(sparse_output=False,handle_unknown='ignore'), onehot_ftrs),
        ('minmax', MinMaxScaler(), minmax_ftrs),
        ('std', StandardScaler(), std_ftrs)])

### Evaluation Functions

In [9]:
def RMSE(y_pred, y_true):
    return np.sqrt(((y_pred - y_true) ** 2).sum() / len(y_pred))

def ALL_ZERO_BASELINE(y_true):
    return np.sqrt(((y_true) ** 2).sum() / len(y_true))

def ALL_MEAN_BASELINE(y_train, y_true):
    mean = y_train.mean()
    return np.sqrt(((mean - y_true) ** 2).sum() / len(y_true))

### Load Trained Models

In [10]:
model_ridge = joblib.load("../results/best_model_ridge.pkl")
model_poly = joblib.load("../results/best_model_ridge_poly.pkl") #this is the better updated poly ridge model
model_SVR = joblib.load("../results/best_model_linearsvr.pkl")
model_RF = joblib.load("../results/best_model_rfr.pkl")
model_XGB = joblib.load("../results/best_model_XGB.pkl")

[18:27:38] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1693020860993/work/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



### Calculate Baselines

In [11]:
LY_BASELINE = RMSE(X_test["TOTAL_POS - lag 1 quarter"], y_test)
AZ_BASELINE = ALL_ZERO_BASELINE(y_test)
AM_BASELINE = ALL_MEAN_BASELINE(y_other, y_test)

### Score Models on Test Set

In [ ]:
y_pred_ridge = model_ridge.predict(X_test)
score_ridge = RMSE(y_pred_ridge, y_test)

y_pred_poly = model_poly.predict(X_test)
score_poly = RMSE(y_pred_poly, y_test)

y_pred_SVR = model_SVR.predict(X_test)
score_SVR = RMSE(y_pred_SVR, y_test)

y_pred_RF = model_RF.predict(X_test)
score_RF = RMSE(y_pred_RF, y_test)

#preprocessor.fit(X_other)
#y_pred_XGB = model_XGB.predict(preprocessor.transform(X_test))
#score_XGB = RMSE(y_pred_XGB, y_test)

### Visualize Results

In [ ]:
data = [
        ['XGBoost', 13.238935694425836],
        ['XGBoost', 11.56658284379576],
        ['XGBoost', 9.815434282889152],
        ['XGBoost', 11.276301956704552],
        ['XGBoost', 10.26521592121377],
        ['LinearSVR', 9.00368299476908],
        ['RFR', 7.478723543421034],
        ['RFR', 7.622241359737829],
        ['RFR', 8.370817501792358],
        ['RFR', 8.20817501792358],
        ['RFR', 8.33719454962583],
        ['Ridge', 8.302734151341832],
        ['Poly', 6.83297972421315],
            ]
tests = pd.DataFrame(data, columns=['model_name', 'model_score'])

In [ ]:
from matplotlib import rc, rcParams
sns.reset_defaults()

fig_title = "model_scores_new.png"
fig_description=""""""

fig = sns.barplot(data=tests, x="model_name", y="model_score")
#fig, ax = plt.subplots()
plt.gca().set_ylim(0, 30)
plt.axhline(AZ_BASELINE, color='black', label='all_zero baseline', linestyle='--')
plt.axhline(LY_BASELINE, color='grey', label='last_year baseline', linestyle='--')

plt.legend()

plt.xlabel('model', fontsize=AXIS_SIZE)
plt.ylabel('Test RMSE [people]', fontsize=AXIS_SIZE)
plt.title('Test Score by Model', fontsize=TITLE_SIZE)

plt.figtext(0.14, -0.12, fig_description, wrap=False, horizontalalignment='left', verticalalignment='top', fontsize=DESCRIPTION_SIZE)
plt.savefig("../figures/" + fig_title, bbox_inches="tight", pad_inches=0.4, dpi=EXPORT_DPI, transparent=False)
plt.show()

## 2. Model Interpretation
### Linear Weights

In [ ]:
def cfs_from(model):
    #gets the coefficients from a model
    cfs = pd.DataFrame(model.named_steps["model"].coef_, columns=['importance'])
    cfs.index = model.named_steps['preprocess'].get_feature_names_out()
    cfs['abs_importance'] = cfs['importance'].abs()
    return cfs.sort_values(by='abs_importance', ascending=False)

In [ ]:
cfs_from(model_ridge)

In [ ]:
cfs_from(model_poly)

In [ ]:
fig_title = "linear-weight-by-feature.png"
fig_description=""""""

data = cfs_from(model_poly).head(20).reindex(AGE_ORDER)
AGE_ORDER = cfs_from(model_poly).head(20).index.to_list()
AGE_ORDER.reverse()
colors = list(map(lambda x: GREEN if x else RED,(data["importance"] > 0)))
data["abs_importance"].plot(kind='barh', figsize=(10*2/3,7*2/3), width=0.65, color=colors)

plt.xlabel('abolute value of coefficient', fontsize=AXIS_SIZE)
plt.ylabel('feature', fontsize=AXIS_SIZE)
plt.title('Weights of Best Linear Model', fontsize=TITLE_SIZE)

plt.figtext(0.14, -0.12, fig_description, wrap=False, horizontalalignment='left', verticalalignment='top', fontsize=DESCRIPTION_SIZE)
plt.savefig("../figures/" + fig_title, bbox_inches="tight", pad_inches=0.4, dpi=400, transparent=False)
plt.show()

### XGBoost Metrics

In [ ]:
XGB = model_XGB
zeros = np.zeros(len(XGB._Booster.get_score(fmap='', importance_type='total_cover')))

metrics = pd.DataFrame(zeros, columns=["0"])

metric_list = ['weight', 'gain', 'cover', 'total_gain', 'total_cover']

for m in metric_list:
    metrics[m] = list(XGB._Booster.get_score(fmap='', importance_type=m).values())

metrics = metrics.drop("0", axis=1)
feature_names = preprocessor.get_feature_names_out()
metrics.index = list(XGB._Booster.get_score(fmap='', importance_type=m).keys())
metrics.index = list(feature_names[[int(e[1:]) for e in metrics.index]])

metrics.head(10)

In [ ]:
for m in metric_list:
    fig_title = "xgb-" + str(m) + ".png"
    fig_description=""""""
    
    #m = 'gain'
    NUM_TO_SHOW = 10
    sorted = metrics.sort_values(by=m, ascending=False)
    plt.figure(figsize=(8, 5))
    plt.barh(sorted.index[:NUM_TO_SHOW][::-1],sorted[m][:NUM_TO_SHOW][::-1])
    
    plt.xlabel(m, fontsize=AXIS_SIZE)
    plt.ylabel('feature', fontsize=AXIS_SIZE)
    plt.title('Features with the Most XGBoost ' + str(m), fontsize=TITLE_SIZE)
    
    plt.figtext(0.14, -0.12, fig_description, wrap=False, horizontalalignment='left', verticalalignment='top', fontsize=DESCRIPTION_SIZE)
    plt.savefig("../figures/" + fig_title, bbox_inches="tight", pad_inches=0.4, dpi=400, transparent=False)
    plt.show()